### 한빛출판사 새로나온책

In [11]:
import requests
from urllib.parse import quote
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import datetime 

In [12]:
# https://www.hanbit.co.kr/store/books/new_book_list.html?page=1&brand=&cate1=&cate2=&searchKey=&keyWord=
base_url = 'https://www.hanbit.co.kr/academy/books/new_book_list.html?page='
sub_url = 'https://www.hanbit.co.kr/academy/books/book_view.html?p_code='
other_url = '&cate_cd=&srt=&searchKey=&keyWord='
header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'}
addr_head = 'https://www.hanbit.co.kr' 

In [13]:
url = f'{base_url}1{other_url}'
result = requests.get(url, headers=header)
soup = BeautifulSoup(result.text, 'html.parser')


In [14]:
lis = soup.select('.sub_book_list_area > li')
len(lis)

20

In [15]:
blist = lis[0]
price = int(blist.select_one('.price').get_text().strip('원').replace(',', ''))
title = blist.select_one('.book_tit > a').get_text().strip()
author = blist.select_one('.book_writer').get_text().strip()
img = addr_head + blist.select_one('.view_box_block > img')['src']


In [32]:
lines = []
for page in tqdm(range(1, 24)):
    url = f'{base_url}{page}{other_url}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')

    lis = soup.select('.sub_book_list_area > li')
    for li in lis:
        price = int(li.select_one('.price').get_text().strip('원').replace(',', ''))
        title = li.select_one('.book_tit > a').get_text().strip()
        author = li.select_one('.book_writer').get_text().strip()
        img = addr_head + li.select_one('.view_box_block > img')['src']
        lines.append({'title': title, 'author': author, 'price': price, 'img': img})


df = pd.DataFrame(lines)
df.head()




100%|██████████| 23/23 [00:06<00:00,  3.82it/s]


,title,author,price,img
0,처음 배우는 매트랩,방성완,30000,https://www.hanbit.co.kr/data/books/B719243717...
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,히든그레이스 데이터분석팀,34000,https://www.hanbit.co.kr/data/books/B911113579...
2,정보교과교육론(3판),"최현종 , 전용주",30000,https://www.hanbit.co.kr/data/books/B698322643...
3,"IT CookBook, 기초전자실험 with PSpice(3판)",홍순관,22000,https://www.hanbit.co.kr/data/books/B543265656...
4,경제수학 강의(3판),김성현,23000,https://www.hanbit.co.kr/data/books/B346928828...


In [17]:
now = datetime.now()
cur_day = now.strftime('%Y%m%d')

df.to_csv(f'data/{cur_day}_한빛아카데미새책.csv', index=False)

- sub page에서 가져오기

In [49]:
lines = []
columns = ('저자 번역 출간 페이지').split()

for page in range(1, 24):
    url = f'{base_url}{page}'
    result = requests.get(url, headers=header)
    soup = BeautifulSoup(result.text, 'html.parser')

    lis = soup.select('li.sub_book_list')
    for li in tqdm(lis):
        ele = {}
        # sub_url = addr_head + li.select_one('.pop_quick_menu > a')['href']
        sub_url = addr_head + li.select_one('.pop_quick_bg')['onclick'][10:-1]
        # sub_res = requests.get(sub_url, headers=header)
        sub_res = requests.get(sub_url)
        sub_soup = BeautifulSoup(sub_res.text, 'html.parser')
        # print('1')

        # 책 제목
        ele['제목'] = sub_soup.select_one('.store_product_info_box > h3').get_text().strip()
        
        ele['번역'] = '-'
        # 저자, 번역, 출간, 페이지 정보 가져오기
        li_list = sub_soup.select('.info_list > li')   

        for li in li_list:
            col = li.find('strong').get_text().strip().strip(' :')

            if col in columns:
                ele[col] = li.find('span').get_text().strip(' 쪽').replace(' , ', ', ')

        # print('3')
        # translator                    
        
        # 가격정보
        ele['가격'] = int(sub_soup.select('.payment_box.curr > p')[1].select('strong')[1].get_text().replace(',',''))
        
        lines.append(ele)
       
df2 = pd.DataFrame(lines)
df2.head(20)


100%|██████████| 13/13 [00:05<00:00,  2.45it/s]


,제목,번역,저자,출간,페이지,가격
0,처음 배우는 매트랩,-,방성완,2023-04-10,416,30000
1,SAS와 메타분석을 활용한 한번에 적용하는 의학보건통계,-,히든그레이스 데이터분석팀,2023-03-09,516,34000
2,정보교과교육론(3판),-,"최현종, 전용주",2023-03-01,488,30000
3,"IT CookBook, 기초전자실험 with PSpice(3판)",-,홍순관,2023-02-28,308,22000
4,경제수학 강의(3판),-,김성현,2023-02-28,464,23000
5,글로벌 사회와 리더십(2판),-,안광현,2023-02-28,416,28000
6,행동 재무론,-,고광수,2023-02-28,596,32000
7,"IT CookBook, 알기 쉽게 해설한 파이썬 with 컴퓨팅 사고력",-,"김충석, 김지연",2023-02-24,328,24000
8,MATLAB으로 배우는 공학 수치해석(3판),-,방성완,2023-02-20,360,28000
9,Cengel의 알기 쉬운 열역학(6판),"김택영, 한진호, 김덕종, 김우태, 이무연, 전문수","Yunus A. Çengel, John M. Cimbala, Afshin J. Gh...",2023-02-17,528,34000


In [50]:
now = datetime.now()
cur_day = now.strftime('%Y%m%d')

df2.to_csv(f'data/{cur_day}_한빛아카데미새책.csv', index=False)